In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
MODEL_NAME = '../meta-llama/Llama-2-7b-chat-hf'

# Load our model from local
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME+'_model', device_map="auto", torch_dtype=torch.float16)

# And its associated tokenizer from local
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME+'_tokenizer')


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [8]:
def get_activation(layer):
    def hook(module, input, output):
        layer.activation = output.detach()  # we use `detach` to separate this value from the computation graph
    return hook

In [24]:
hook_ref = model.lm_head.register_forward_hook(get_activation(model.lm_head))
input = tokenizer("This is a test sentence", return_tensors="pt").to(device)
outputs = model(**input)  # Process the input
activation = model.lm_head.activation  # Access the activation of the lm_head layer you just hooked
hook_ref.remove()

In [25]:
activation

tensor([[[  0.1039,  -0.2219,   0.3132,  ...,   1.3271,   1.8799,   0.6436],
         [-10.1562,  -4.5742,  -4.0391,  ...,  -5.3633,  -6.7422,  -6.3516],
         [ -5.2188,  -2.1855,   0.5386,  ...,  -1.1602,  -2.3125,  -4.0938],
         [ -6.5625,  -3.0293,  -2.2617,  ...,  -2.8516,  -4.3047,  -3.6836],
         [ -7.6523,  -1.6934,   1.0615,  ...,  -7.2305,  -5.2891,  -7.9023],
         [ -2.7520,   4.0352,  11.9297,  ...,  -1.0010,  -2.3770,  -1.8301]]],
       device='cuda:0', dtype=torch.float16)

In [26]:
activation.shape

torch.Size([1, 6, 32000])

In [51]:
hook_ref = model.model.layers[0].mlp.register_forward_hook(get_activation(model.model.layers[0].mlp))
inputs = tokenizer("how ar eyou", return_tensors="pt").to(device)
outputs = model(**inputs)
activation = model.model.layers[0].mlp.activation
hook_ref.remove()

In [39]:
activation.shape

torch.Size([1, 9, 4096])

In [52]:
# Assume 'output' is the final output from your model
output_predictions = outputs.logits

# Get the predicted token ids
predicted_ids = torch.argmax(output_predictions, dim=-1)

# Convert token ids back to words
predicted_sentence = tokenizer.decode(predicted_ids[0])

predicted_sentence

'Unterscheidung torogyou doing'

In [65]:
def top_k_sampling(logits, k=5):
    indices_to_remove = logits < torch.topk(logits, k)[0][..., -1, None]
    logits[indices_to_remove] = float('-inf')
    probs = torch.nn.functional.softmax(logits, dim=-1)
    next_token = torch.multinomial(probs, num_samples=1)
    return next_token

input_sentence = "what color is an apple"
inputs = tokenizer.encode(input_sentence, return_tensors="pt").to(device)
generated_sentence = inputs

model.eval()
with torch.no_grad():
    for _ in range(100):
        outputs = model(generated_sentence)
        next_token_logits = outputs.logits[:, -1, :]
        next_token = top_k_sampling(next_token_logits)
        generated_sentence = torch.cat((generated_sentence, next_token), dim=1)

print(tokenizer.decode(generated_sentence[0]))

<s> what color is an apple?

Answer: An apple is typically a deep red color, although some varieties may be yellow, green, or a combination of colors.

Here are some fun facts about apples:

1. Apples are a type of fruit that belong to the rose family.
2. There are over 7,500 varieties of apples worldwide.
3. Apples are one of the most widely consumed fruits in the world, with over


In [ ]:
numpy_array = hooked_layer.activation.detach().cpu().numpy()
# Save the numpy array
np.save('activation.npy', numpy_array)